<a href="https://colab.research.google.com/github/snehamurali35/MuraliUW/blob/main/Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import requests
import json
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
def get_inrix_data():
  headers = {
      'Authorization': 'Bearer '  + 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJhcHBJZCI6IjdzM2QxdmF5cGYiLCJ0b2tlbiI6eyJpdiI6IjhhNjZjMmYxNTgwYTVhNzBhODVmODVmMTk1MjVhMjBkIiwiY29udGVudCI6IjBjMjdlNmZiNTg1N2NmMzdhMDEwZDVjZGNmYTllM2NmMzRkYzdiMzk4MTNlZWI2MjM1YzVkMzY3YWEzMWM3MTEzY2M4ODA1OGY2ZmQ1ZGQ2YjNlMzg1MGQ5ZWRhNDdmMjZhYmY4YzRkYjQxYjJmMDIwMWFjOWFjOGU2ZTNiZjU2MDk3N2I1NGQ4Zjg0ZDYxNDY2NmFlNTg5YzY4YWE3YjQ5MWJhYTM1ZDBjNDMzMWJhZThkYzAwMGVjY2U3MzViMjA4NDdlZDM3NjczY2VjODU0ZTNkNzNlOTYyODc2YTAwNTdjM2MxOGRlMzdjZGM5ZmI4MDJlYjdkYWE5ZTkwNDQwNjQzM2E5MGYzYjZmYTRjZWNkMjAzNzIwZDdlNjY2ODA4MmFjMDQ0MTZlNDAzNGJlOTc2ZTQ3ZmNlMjMyNjE1NDE4ZDQyMmI5YmU3NzExZWQ2N2RlYTkxMmZkYTI3YjQ4NzFiOGNmMjkxNTQ1MjVjN2M4NjAxOWEyNzQ3N2U4Y2EyYjI1MzQ0ODA0NTE4ZWI5NzY1ZmNmYzljYmZmYjNmMTFmZGE3NmZmMzljMWE5NTlkNDJhNmZkOGY2ZjY5NzAzMzIxNjI0ZTJkNzljZDg1ZGQ3MzM5ZGMyMTc1YWY4OTkyNWEyOTUzNTk5ZWZmYjE3MTJlODFmYzkxMzI2ZmE2ZTYyZWJjODI4ZTg4NWE4Y2UwODc2NWYwOGRhOGViMjU4ODE5ZGI2MGQwYWUzOTgyZjRkYmE0OWM4OTVhYjA2YzZhM2QzNGUxOGIyN2FmMTk3MzM1MzkwNTM4NDRmZGJmMzZkZWQ4MjFjZTE4N2YyOWEyMzE4YzNhM2Q1ZjkxNTczMDk2ZWFlM2RlMWI3ODcyODhiNzExIn0sInNlY3VyaXR5VG9rZW4iOnsiaXYiOiI4YTY2YzJmMTU4MGE1YTcwYTg1Zjg1ZjE5NTI1YTIwZCIsImNvbnRlbnQiOiIwYTA5ZmJjMTZkNThjZTBmZGUwYWFiYjZmZmU2Y2VhNDBmYzY3YTJlYTAyOWRjNjIxM2ZmZTgwMjg2MGZlMDA3MDRlNjgzMTZlYWNlNDVkNGFiYjRmNTMzIn0sImp0aSI6ImYzNDI4OWQ2LTFlY2EtNGRkZS05YmIzLWMxYjkzYzY3MzVkMiIsImlhdCI6MTcwMTYyMjkzNiwiZXhwIjoxNzAxNjI2NTM2fQ.tf8pZTGG9gMW4zB6HglMQQKOPx0tuC9lzvOSWBFpAyQ',
      'Content-Type': 'application/json',
  }

  df = pd.DataFrame([], columns = ['Average Speed', 'Speed Limit', 'Congestion'])

  try:
      response = requests.get('https://api.iq.inrix.com/v1/segments/speed?box=37.757386%7C-122.490667%2C37.746138%7C-122.395481&SpeedOutputFields=All', headers=headers)
      response.raise_for_status()  # Raise an exception for bad responses

      # Assuming the API response is in JSON format
      api_data = response.json()
      #api_data = json.loads(api_data)
      #print(api_data.get("result").get("segmentspeeds")[0].get("segments"))

      # Save the JSON data to a file
      for segment in api_data.get("result").get("segmentspeeds")[0].get("segments"):
        average = float(segment["average"])
        speedlimit = float(segment["reference"])
        congestion = 1 - (average / speedlimit)
        df.loc[len(df.index)] = [average, speedlimit, congestion]

  except requests.exceptions.RequestException as e:
    print(f"Error making API call: {e}")

  return df

In [ ]:
def make_neural_network():
    data = get_inrix_data()

    # Selecting features (X) and target variable (y)
    X = data[['Average Speed', 'Speed Limit']].astype(float)
    y = data['Congestion']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Standardize features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Build the neural network model
    model = Sequential()
    model.add(Dense(units=64, activation='relu', input_dim=2))  # 2 input features
    model.add(Dense(units=32, activation='relu'))
    model.add(Dense(units=1, activation='linear'))  # 1 output for regression

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train_scaled, y_train, epochs=50, batch_size=2, verbose=1)

    # Evaluate the model on the test set
    y_pred = model.predict(X_test_scaled)

    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error: {mse}")

    # test a new data point
    for i in [20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]:
      x_new = pd.DataFrame([[65, i]], columns = ['Average Speed', 'Speed Limit'])
      x_new_scaled = scaler.transform(x_new)
      y_new = model.predict(x_new_scaled)
      print(i, y_new)

In [ ]:
def main():
  make_neural_network()

if __name__ == "__main__":
    main()

Epoch 1/50
187/187 [==============================] - 1s 2ms/step - loss: 0.0046
Epoch 2/50
187/187 [==============================] - 0s 2ms/step - loss: 2.3181e-04
Epoch 3/50
187/187 [==============================] - 0s 3ms/step - loss: 7.0055e-04
Epoch 4/50
187/187 [==============================] - 1s 3ms/step - loss: 3.0219e-04
Epoch 5/50
187/187 [==============================] - 1s 3ms/step - loss: 0.0011
Epoch 6/50
187/187 [==============================] - 1s 3ms/step - loss: 0.0047
Epoch 7/50
187/187 [==============================] - 1s 3ms/step - loss: 1.8312e-04
Epoch 8/50
187/187 [==============================] - 1s 3ms/step - loss: 3.8250e-05
Epoch 9/50
187/187 [==============================] - 1s 3ms/step - loss: 5.2712e-05
Epoch 10/50
187/187 [==============================] - 1s 3ms/step - loss: 2.8286e-05
Epoch 11/50
187/187 [==============================] - 1s 3ms/step - loss: 3.7401e-05
Epoch 12/50
187/187 [==============================] - 0s 2ms/step - loss: 